<a href="https://colab.research.google.com/github/Nathan-Ketterlinus/NavierStokesAlphaNN/blob/main/NK_Adjusted_NSA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepxde

In [ ]:
import deepxde as dde

In [1]:
from functools import partial
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

#-----------------------------------------------------------#
# TO DO                                                     #
#-----------------------------------------------------------#
# 1. add nudging term (eta) to f1, f2, f3                   #
# 2. test fn at various eta values (0.1, 10, 30)            #
# 3. update graph to show differences across eta, not alpha #
#-----------------------------------------------------------#

# -----------------------------------
# These are the values you need to change
# -----------------------------------
alpha = 0.5 # filtering strength; when (a=0), (v=u)
rho = 1.0
mu = 1.0
eta = 0.1
# -----------------------------------

# Navier-Stokes PDE
def ns_pde(x, u):

    # ns_pde takes:
      # x: input coordinate tensor [x, y, z, t]
      # u: output velocity tensor [u_vel, v_vel, w_vel, p]
    # and returns:
      # momentum_x: momentum on x-axis @ input point x
      # momentum_y: momentum on y-axis @ input point x
      # momentum_z: momentum on z-axis @ input point x
      # continuity: divergence = 0 condition for NS

    u_vel, v_vel, w_vel, p = u[:, 0:1], u[:, 1:2], u[:, 2:3], u[:, 3:4]

    # external forces on system (gravity, etc)
    f_x, f_y, f_z = 0.0, 0.0, 0.0

    # First Derivatives
    # Jacobian takes:
      # u: output tensor [u_vel, v_vel, w_vel, p]
      # x: input tensor [x, y, z, t]
      # i: index of u to use (u[i])
      # j: index of x to use (x[j])
    # and returns:
      # (partial u[i]) / (partial x[j])
    u_vel_t = dde.grad.jacobian(u, x, i=0, j=3)  # du/dt
    u_vel_x = dde.grad.jacobian(u, x, i=0, j=0)  # du/dx
    u_vel_y = dde.grad.jacobian(u, x, i=0, j=1)  # du/dy
    u_vel_z = dde.grad.jacobian(u, x, i=0, j=2)  # du/dz

    v_vel_t = dde.grad.jacobian(u, x, i=1, j=3)  # dv/dt
    v_vel_x = dde.grad.jacobian(u, x, i=1, j=0)  # dv/dx
    v_vel_y = dde.grad.jacobian(u, x, i=1, j=1)  # dv/dy
    v_vel_z = dde.grad.jacobian(u, x, i=1, j=2)  # dv/dz

    w_vel_t = dde.grad.jacobian(u, x, i=2, j=3)  # dw/dt
    w_vel_x = dde.grad.jacobian(u, x, i=2, j=0)  # dw/dx
    w_vel_y = dde.grad.jacobian(u, x, i=2, j=1)  # dw/dy
    w_vel_z = dde.grad.jacobian(u, x, i=2, j=2)  # dw/dz

    p_x = dde.grad.jacobian(u, x, i=3, j=0)  # dp/dx
    p_y = dde.grad.jacobian(u, x, i=3, j=1)  # dp/dy
    p_z = dde.grad.jacobian(u, x, i=3, j=2)  # dp/dz

    # Second Derivatives
    # Hessian takes:
      # u: output tensor [u_vel, v_vel, w_vel, p]
      # x: input tensor [x, y, z, t]
      # component: index of u[] to differentiate (u[component])
      # i: index of x to differentiate w.r.t 1st (x[i])
      # j: index of x to differentiate w.r.t 2nd (x[j])
    # and returns:
      # (∂^2 u[component]) / (∂x[i] ∂x[j])
    u_vel_xx = dde.grad.hessian(u, x, component=0, i=0, j=0)  # d²u/dx²
    u_vel_yy = dde.grad.hessian(u, x, component=0, i=1, j=1)  # d²u/dy²
    u_vel_zz = dde.grad.hessian(u, x, component=0, i=2, j=2)  # d²u/dz²

    v_vel_xx = dde.grad.hessian(u, x, component=1, i=0, j=0)  # d²v/dx²
    v_vel_yy = dde.grad.hessian(u, x, component=1, i=1, j=1)  # d²v/dy²
    v_vel_zz = dde.grad.hessian(u, x, component=1, i=2, j=2)  # d²v/dz²

    w_vel_xx = dde.grad.hessian(u, x, component=2, i=0, j=0)  # d²w/dx²
    w_vel_yy = dde.grad.hessian(u, x, component=2, i=1, j=1)  # d²w/dy²
    w_vel_zz = dde.grad.hessian(u, x, component=2, i=2, j=2)  # d²w/dz²

    # (Should be of the form ∂u/∂t + (u⋅∇)u + ∇p - νΔu - f = 0)
    # NS-x
    momentum_x = (
        rho * (u_vel_t + (u_vel * u_vel_x) + (v_vel * u_vel_y) + (w_vel * u_vel_z)) # ∂u/∂t + ((u⋅∇)u)_x
        - rho * f_x + p_x                                                           # -f_x, +∇p_x
        - mu * (u_vel_xx + u_vel_yy + u_vel_zz)                                     # -νΔu_x
    )

    # NS-y
    momentum_y = (
        rho * (v_vel_t + (u_vel * v_vel_x) + (v_vel * v_vel_y) + (w_vel * v_vel_z)) # ∂u/∂t + ((u⋅∇)u)_y
        - rho * f_y + p_y                                                           # -f_y, +∇p_y
        - mu * (v_vel_xx + v_vel_yy + v_vel_zz)                                     # -νΔu_y
    )

    # NS-z
    momentum_z = (
        rho * (w_vel_t + (u_vel * w_vel_x) + (v_vel * w_vel_y) + (w_vel * w_vel_z)) # ∂u/∂t + ((u⋅∇)u)_y
        - rho * f_z + p_z                                                           # -f_z, +∇p_z
        - mu * (w_vel_xx + w_vel_yy + w_vel_zz)                                     # -νΔu_z
    )

    # continuity (∇⋅u = 0)
    continuity = u_vel_x + v_vel_y + w_vel_z

    # We return these to have the NN learn to set them all = 0
    # for arbitrary input values, thus adhering to the NS equations2
    return [momentum_x, momentum_y, momentum_z, continuity]

def ns_alpha_pde(x, y):

    # ns_alpha_pde function takes:
      # x: input tensor [x, y, z, t]
      # y: output tensor [u, v, w, u_bar, v_bar, w_bar, q]
    # and returns
      # f1: momentum in x-direction
      # f2: momentum in y-direction
      # f3: momentum in z-direction
      # incompressibility: Divergence should
      # relation_u: how u_bar relates to u
      # relation_v: how v_bar relates to v
      # relation_w: how w_bar relates to w
    # These return values are what the NN is trying to minimize the loss of


    #external forces on x, y, z axis
    f_x = 0
    f_y = 0
    f_z = 0

    # Components of our output (y)
    # Each component is a column vector ([N,1])
    u, v, w, u_bar, v_bar, w_bar, q = y[:, 0:1], y[:, 1:2], y[:, 2:3], y[:, 3:4], y[:, 4:5], y[:, 5:6], y[:, 6:7]

    # First-order derivatives
    # Jacobian takes:
      # y: output tensor [u, v, w, u_bar, v_bar, w_bar, q]
      # x: input tensor [x, y, z, t]
      # i: index of y to use (y[i])
      # j: index of x to use (x[j])
    # and returns:
      # (∂y[i]) / (∂x[j])

    du_t = dde.grad.jacobian(y, x, i=0, j=3)  # ∂u/∂t
    dv_t = dde.grad.jacobian(y, x, i=1, j=3)  # ∂v/∂t
    dw_t = dde.grad.jacobian(y, x, i=2, j=3)  # ∂w/∂t

    ### ADDED FOR CHANGED f1, f2, f3 DEFINITIONS ####
    du_x = dde.grad.jacobian(y, x, i=0, j=0)  # ∂u/∂x
    du_y = dde.grad.jacobian(y, x, i=0, j=1)  # ∂u/∂y
    du_z = dde.grad.jacobian(y, x, i=0, j=2)  # ∂u/∂z

    dv_x = dde.grad.jacobian(y, x, i=1, j=0)  # ∂v/∂x
    dv_y = dde.grad.jacobian(y, x, i=1, j=1)  # ∂v/∂y
    dv_z = dde.grad.jacobian(y, x, i=1, j=2)  # ∂v/∂z

    dw_x = dde.grad.jacobian(y, x, i=2, j=0)  # ∂w/∂x
    dw_y = dde.grad.jacobian(y, x, i=2, j=1)  # ∂w/∂y
    dw_z = dde.grad.jacobian(y, x, i=2, j=2)  # ∂w/∂z
    #################################################

    du_bar_x = dde.grad.jacobian(y, x, i=3, j=0)  # ∂u_bar/∂x
    du_bar_y = dde.grad.jacobian(y, x, i=3, j=1)  # ∂u_bar/∂y
    du_bar_z = dde.grad.jacobian(y, x, i=3, j=2)  # ∂u_bar/∂z
    du_bar_t = dde.grad.jacobian(y, x, i=3, j=3)  # ∂u_bar/∂t

    dv_bar_x = dde.grad.jacobian(y, x, i=4, j=0)  # ∂v_bar/∂x
    dv_bar_y = dde.grad.jacobian(y, x, i=4, j=1)  # ∂v_bar/∂y
    dv_bar_z = dde.grad.jacobian(y, x, i=4, j=2)  # ∂v_bar/∂z
    dv_bar_t = dde.grad.jacobian(y, x, i=4, j=3)  # ∂u_bar/∂t

    dw_bar_x = dde.grad.jacobian(y, x, i=5, j=0)  # ∂w_bar/∂x
    dw_bar_y = dde.grad.jacobian(y, x, i=5, j=1)  # ∂w_bar/∂y
    dw_bar_z = dde.grad.jacobian(y, x, i=5, j=2)  # ∂w_bar/∂z
    dw_bar_t = dde.grad.jacobian(y, x, i=5, j=3)  # ∂u_bar/∂t

    # (for the Pressure Gradient (nabla p))
    dq_x = dde.grad.jacobian(y, x, i=6, j=0)  # ∂q/∂x
    dq_y = dde.grad.jacobian(y, x, i=6, j=1)  # ∂q/∂y
    dq_z = dde.grad.jacobian(y, x, i=6, j=2)  # ∂q/∂z

    # Second-order derivatives
    # Hessian takes:
      # y: output tensor [u, v, w, u_bar, v_bar, w_bar, q]
      # x: input tensor [x, y, z, t]
      # component: index of y[] to differentiate (y[component])
      # i: index of x to differentiate w.r.t 1st (x[i])
      # j: index of x to differentiate w.r.t 2nd (x[j])
    # and returns:
      # (∂^2 y[component]) / (∂x[i] ∂x[j])

    du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0) # ∂u/∂x^2
    du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1) # ∂u/∂y^2
    du_zz = dde.grad.hessian(y, x, component=0, i=2, j=2) # ∂u/∂z^2

    dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0) # ∂v/∂x^2
    dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1) # ∂v/∂y^2
    dv_zz = dde.grad.hessian(y, x, component=1, i=2, j=2) # ∂v/∂z^2

    dw_xx = dde.grad.hessian(y, x, component=2, i=0, j=0) # ∂w/∂x^2
    dw_yy = dde.grad.hessian(y, x, component=2, i=1, j=1) # ∂w/∂y^2
    dw_zz = dde.grad.hessian(y, x, component=2, i=2, j=2) # ∂w/∂z^2

    du_bar_xx = dde.grad.hessian(y, x, component=3, i=0, j=0) # ∂u_bar/∂x^2
    du_bar_yy = dde.grad.hessian(y, x, component=3, i=1, j=1) # ∂u_bar/∂y^2
    du_bar_zz = dde.grad.hessian(y, x, component=3, i=2, j=2) # ∂u_bar/∂z^2

    dv_bar_xx = dde.grad.hessian(y, x, component=4, i=0, j=0) # ∂v_bar/∂x^2
    dv_bar_yy = dde.grad.hessian(y, x, component=4, i=1, j=1) # ∂v_bar/∂y^2
    dv_bar_zz = dde.grad.hessian(y, x, component=4, i=2, j=2) # ∂v_bar/∂z^2

    dw_bar_xx = dde.grad.hessian(y, x, component=5, i=0, j=0) # ∂w_bar/∂x^2
    dw_bar_yy = dde.grad.hessian(y, x, component=5, i=1, j=1) # ∂w_bar/∂y^2
    dw_bar_zz = dde.grad.hessian(y, x, component=5, i=2, j=2) # ∂w_bar/∂z^2

    # Laplacians
    # (used in Diffusion (nu Δ v))
    laplacian_u = du_xx + du_yy + du_zz
    laplacian_v = dv_xx + dv_yy + dv_zz
    laplacian_w = dw_xx + dw_yy + dw_zz

    laplacian_u_bar = du_bar_xx + du_bar_yy + du_bar_zz
    laplacian_v_bar = dv_bar_xx + dv_bar_yy + dv_bar_zz
    laplacian_w_bar = dw_bar_xx + dw_bar_yy + dw_bar_zz

    # Relationship between filtered and actual velocities
    # (v = (1-α^2 Δ)u)
    # this forces our model to learn how to be constrained to the proper
    # relationship between our velocity field and our filtered velocity field

    ### INVERTING RELATIONSHIP BETWEEN u and V ###########
    #relation_u = u_bar - alpha**2 * laplacian_u_bar - u #
    #relation_v = v_bar - alpha**2 * laplacian_v_bar - v #
    #relation_w = w_bar - alpha**2 * laplacian_w_bar - w #
    ######################################################

    relation_u = u - alpha**2 * laplacian_u - u_bar
    relation_v = v - alpha**2 * laplacian_v - v_bar
    relation_w = w - alpha**2 * laplacian_w - w_bar

    # Momentum equations
    # (of the form ∂u/∂t - ū×(∇×ū) + ∇q - νΔu - f = 0)

    # NOTE: Viscous term is on our unfiltered velocity field u, NOT the filtered u_bar
    # this SHOULD be fine since we're also enforcing the filtering constraint
    # u_bar = (1 − α^2 Δ)u

    # u = UNFILTERED
    # u_bar = FILTERED

    f1 = (
        du_bar_t                                                  # ∂u_bar/∂t
        - (v * (dv_bar_x - du_bar_y) - w * (du_bar_z - dw_bar_x)) # x-component of u × (∇ × u_bar)
        + dq_x                                                    # pressure gradient (∂p/∂x)
        - mu * laplacian_u_bar                                    # viscosity term on u: -mu Δu_bar (changed 7/5)
        - f_x                                                     # external force in x-direction
      )

    f2 = (
        dv_bar_t                                                  # ∂v_bar/∂t
        - (w * (dw_bar_y - dv_bar_z) - u * (dv_bar_x - du_bar_y)) # y-component of u × (∇ × u_bar)
        + dq_y                                                    # pressure gradient (∂p/∂y)
        - mu * laplacian_v_bar                                    # viscosity term on v: -mu Δu_bar (changed 7/5)
        - f_y                                                     # external force in y-direction
    )

    f3 = (
        dw_bar_t                                                  # ∂w_bar/∂t
        - (u * (du_bar_z - dw_bar_x) - v * (dw_bar_y - dv_bar_z)) # z-component of u × (∇ × u_bar)
        + dq_z                                                    # pressure gradient (∂p/∂y)
        - mu * laplacian_w_bar                                    # Viscosity term on w: -mu Δu_bar (changed 7/5)
        - f_z                                                     # external force in z-direction
    )

    # Incompressibility condition for bar velocities
    # ∇ * u = 0

    incompressibility = du_x + dv_y + dw_z

    # Tell NN to minimize: 3 momentum residuals (the filtered equation),
	  # the incompressibility condition,
	  # & 3 filtering constraint residuals
    return [f1, f2, f3, incompressibility, relation_u, relation_v, relation_w]

# Create gridfor
x, y, z = np.meshgrid(
    np.linspace(-1, 1, 10),
    np.linspace(-1, 1, 10),
    np.linspace(-1, 1, 10)
)
X = np.vstack((np.ravel(x), np.ravel(y), np.ravel(z))).T
t_0 = np.zeros((X.shape[0], 1))  # Initial time = 0


# Geometry and time domain
geom = dde.geometry.Cuboid([-1, -1, -1], [1, 1, 1])  # 3D spatial domain
timedomain = dde.geometry.TimeDomain(0, 1)           # Time domain
geomtime = dde.geometry.GeometryXTime(geom, timedomain)  # Combine spatial and temporal domains


# Define periodic boundary conditions
def periodic_bc(x, on_boundary):
    x_spatial = x[:3]  # Just x, y, z (exclude time)
    return on_boundary and (
        np.isclose(x_spatial[0], -1) or np.isclose(x_spatial[0], 1) or
        np.isclose(x_spatial[1], -1) or np.isclose(x_spatial[1], 1) or
        np.isclose(x_spatial[2], -1) or np.isclose(x_spatial[2], 1)
    )


# Create periodic boundary conditions for velocity components
bc_u1 = dde.icbc.PeriodicBC(geomtime, 0, periodic_bc, component=0)  # u
bc_u2 = dde.icbc.PeriodicBC(geomtime, 1, periodic_bc, component=1)  # v
bc_u3 = dde.icbc.PeriodicBC(geomtime, 2, periodic_bc, component=2)  # w

# Create periodic boundary conditions for filtered velocity components
bc_u_bar = dde.icbc.PeriodicBC(geomtime, 0, periodic_bc, component=3)  # u_bar
bc_v_bar = dde.icbc.PeriodicBC(geomtime, 1, periodic_bc, component=4)  # v_bar
bc_w_bar = dde.icbc.PeriodicBC(geomtime, 2, periodic_bc, component=5)  # w_bar

# Define Navier-Stokes model
data_ns = dde.data.TimePDE(
    geomtime, ns_pde,
    [bc_u1, bc_u2, bc_u3],
    num_domain=500, num_boundary=50
)
net_ns = dde.nn.FNN([4] + [50] * 3 + [4], "tanh", "Glorot normal")
model_ns = dde.Model(data_ns, net_ns)
model_ns.compile("adam", lr=1e-3)
losshistory_ns, train_state_ns = model_ns.train(iterations=10000)

#define these for plots at the bottom:
#saves current alpha value
alpha_values = []
#saves diffarr for current alpha value
alphadiffarr = []

# LOOP OVER NS-A MODEL FOR SEVERAL a VALUES
while alpha <= 100:
  print(f"Training on current alpha value: {alpha}")
  # Define NS-alpha model

  # Define training data:
  data_nsa = dde.data.TimePDE(
      geomtime, ns_alpha_pde, # problem domain, pde model
      [bc_u1, bc_u2, bc_u3, bc_u_bar, bc_v_bar, bc_w_bar],  # Corrected boundary conditions ()
      num_domain=500, # sample <num_domain> random points in our domain and train on them (500 unique [x, y, z, t] EACH EPOCH)
      num_boundary=50 # sample an additional <num_boundary> random points ON our domain BOUNDARY (50 unique [x, y, z, t] EACH EPOCH)
  )

  # Defines our network as:
    # Input: 4 nodes (x, y, z, t)
    # 3 hidden layers with 50 nodes each
    # Output: 7 nodes (u, v, w, u_bar, v_bar, w_bar, q)
    # tanh: activation function
    # Glorot normal: weight initializer
  net_nsa = dde.nn.FNN([4] + [50] * 3 + [7], "tanh", "Glorot normal")

  # create model with
    # data_nsa: training data to use
    # net_nsa: network to use
  model_nsa = dde.Model(data_nsa, net_nsa)

  #compile with:
    # optimizer: adam (Adaptive momentum)
    # learning rate: lr = 1e-3
  model_nsa.compile("adam", lr=1e-3)

  # train for 10000 epochs, & store loss to <losshistory_nsa>
  losshistory_nsa, train_state_nsa = model_nsa.train(iterations=10000)

  # Solve Navier-Stokes model
  output_ns = model_ns.predict(np.hstack((X, t_0)))
  u_ns, v_ns, w_ns = output_ns[:, 0], output_ns[:, 1], output_ns[:, 2] # store the output of u, v, w

  # Solve NS-alpha model
  output_nsa = model_nsa.predict(np.hstack((X, t_0)))
  u_nsa, v_nsa, w_nsa = output_nsa[:, 0], output_nsa[:, 1], output_nsa[:, 2] # store the output of u, v, w (SHOULD THIS BE u_bar, v_bar, w_bar??)

  # Compute velocity differences
  u_diff = u_ns - u_nsa
  v_diff = v_ns - v_nsa
  w_diff = w_ns - w_nsa

  # Function to compute L2 norm
  def compute_l2_norm(u_diff, v_diff, w_diff):
      return np.sqrt(np.sum(u_diff**2 + v_diff**2 + w_diff**2))

  # Compare results over time
  plotarr = []
  diffarray = []

  for t in np.linspace(0, 1, 50):  # Multiple time steps
      t_val = np.ones((X.shape[0], 1)) * t
      output_ns = model_ns.predict(np.hstack((X, t_val)))
      output_nsa = model_nsa.predict(np.hstack((X, t_val)))

      u_ns, v_ns, w_ns = output_ns[:, 0], output_ns[:, 1], output_ns[:, 2]
      u_nsa, v_nsa, w_nsa = output_nsa[:, 0], output_nsa[:, 1], output_nsa[:, 2]

      # Compute velocity differences at each time step
      u_diff = u_ns - u_nsa
      v_diff = v_ns - v_nsa
      w_diff = w_ns - w_nsa

      # Calculate L2 norm
      l2_norm = compute_l2_norm(u_diff, v_diff, w_diff)
      plotarr.append(t)
      diffarray.append(l2_norm)


  # Plot the L2 norm of velocity differences over time
  plt.plot(plotarr, diffarray)
  plt.xlabel("Time (s)")
  plt.ylabel("L2 Norm of Velocity Difference")
  plt.title(f"Velocity Difference between NS and NS-alpha Solutions (alpha={alpha:.5g})")
  plt.show()

  #save current diffarr to array for graph of several a values
  alpha_values.append(alpha)
  alphadiffarr.append(diffarray)

  #increment alpha for next loop
  alpha *= 10

# END OF LOOP

# Plot average L2 norm of velocity across several alpha values
plt.figure()
plt.boxplot(alphadiffarr)
plt.xticks(
    ticks=np.arange(1, len(alpha_values) + 1),
    labels=[f"{a:.5g}" for a in alpha_values]
)
plt.xlabel("Alpha value")
plt.ylabel("L2 Norm of Velocity Difference")
plt.title("Velocity Difference between NS and NS-alpha Solutions across several alpha values")
plt.show()

ModuleNotFoundError: No module named 'deepxde'